In [2]:
import pandas as pd
import numpy as np

# Getting the Rate Data

In [156]:
year = "2012"

data_header_list = ["id", "tweet_time","team","opponent", "week", "home_or_away","score","opponent_score","point_spread","over_under"]
data_df = pd.read_csv('nfl-tweets-v1.0/tweets.nfl.'+year+'.weekly.csv', names=data_header_list)

In [157]:
df = data_df.groupby(['team','week'],as_index=False).count()
df_counts = df.filter(['team','week','id'], axis=1)
df_counts = df_counts.rename({'id': 'volume'}, axis=1)
df_counts["rate"] = np.nan
df_counts

,team,week,volume,rate
0,ARI,1,3998,NaN
1,ARI,2,1414,NaN
2,ARI,3,1643,NaN
3,ARI,4,1374,NaN
4,ARI,5,2167,NaN
...,...,...,...,...
475,WAS,12,1564,NaN
476,WAS,13,3313,NaN
477,WAS,14,1912,NaN
478,WAS,15,2045,NaN


In [158]:
# this is where the magic happens to calculate rates
theta = 0.2
for team in mappings.keys():
    rows = df_counts.index[df["team"] == team].tolist()
    df_counts.loc[rows[0],'rate'] = 0
    for r in rows[1:]:
        curr = df_counts.iloc[r]['volume']
        old = df_counts.iloc[r-1]['volume']
        rate = (curr-old)/ (theta*old)
        df_counts.loc[r,'rate'] = rate
        

In [159]:
df_counts

,team,week,volume,rate
0,ARI,1,3998,0.000000
1,ARI,2,1414,-3.231616
2,ARI,3,1643,0.809760
3,ARI,4,1374,-0.818624
4,ARI,5,2167,2.885735
...,...,...,...,...
475,WAS,12,1564,-1.155359
476,WAS,13,3313,5.591432
477,WAS,14,1912,-2.114398
478,WAS,15,2045,0.347803


# Adding Twitter Sentiment to the Data

In [62]:
mappings = {"ARI":"crd", "ATL":"atl","BAL":"rav","BUF":"buf","CAR":"car","CHI":"chi","CIN":"cin","CLE":"cle","DAL":"dal","DEN":"den","DET":"det","GB":"gnb","HOU":"htx","IND":"clt","JAX":"jax","KC":"kan","MIA":"mia","MIN":"min","NE":"nwe","NO":"nor","NYG":"nyg","NYJ":"nyj","OAK":"rai","PHI":"phi","PIT":"pit","SD":"sdg","SEA":"sea","SF":"sfo","STL":"ram","TB":"tam","TEN":"oti","WAS":"was"}
mappings_rev = dict((v,k) for k,v in mappings.items())

{'crd': 'ARI',
 'atl': 'ATL',
 'rav': 'BAL',
 'buf': 'BUF',
 'car': 'CAR',
 'chi': 'CHI',
 'cin': 'CIN',
 'cle': 'CLE',
 'dal': 'DAL',
 'den': 'DEN',
 'det': 'DET',
 'gnb': 'GB',
 'htx': 'HOU',
 'clt': 'IND',
 'jax': 'JAX',
 'kan': 'KC',
 'mia': 'MIA',
 'min': 'MIN',
 'nwe': 'NE',
 'nor': 'NO',
 'nyg': 'NYG',
 'nyj': 'NYJ',
 'rai': 'OAK',
 'phi': 'PHI',
 'pit': 'PIT',
 'sdg': 'SD',
 'sea': 'SEA',
 'sfo': 'SF',
 'ram': 'STL',
 'tam': 'TB',
 'oti': 'TEN',
 'was': 'WAS'}

In [160]:
df = pd.read_csv("2012_examples_teams.csv")

In [161]:
df["Home Twitter Sentiment"] = np.nan
df["Away Twitter Sentiment"] = np.nan
df["Home Twitter Rate"] = np.nan
df["Away Twitter Rate"] = np.nan

In [162]:
twitter = pd.read_csv("2012sentiment.csv")
weeks = set(df["Week"].tolist())
for w in weeks:
    rows = df.index[df["Week"] == w].tolist()
    for r in rows:
        home = mappings_rev[df.iloc[r]["Home Team"]]
        away = mappings_rev[df.iloc[r]["Away Team"]]
        home_sent = twitter.query('team == "'+home+'" and week =='+str(w)+'')
        away_sent = twitter.query('team == "'+away+'" and week =='+str(w)+'')
        home_rate = df_counts.query('team == "'+home+'" and week =='+str(w)+'')
        away_rate = df_counts.query('team == "'+away+'" and week =='+str(w)+'')
        df.loc[r, 'Home Twitter Sentiment'] = home_sent["sentiment"].values[0]
        df.loc[r, 'Away Twitter Sentiment'] = away_sent["sentiment"].values[0]
        df.loc[r, 'Home Twitter Rate'] = home_rate["rate"].values[0]
        df.loc[r, 'Away Twitter Rate'] = away_rate["rate"].values[0]
    

In [163]:
df = df.drop(columns=['Home Team',"Away Team","Week"])


In [164]:
#data2010 = df
#data2011 = df
data2012 = df

In [168]:
df = pd.concat([data2010,data2011,data2012])

KeyError: 0

In [170]:
df.to_csv("all_years_twitter.csv",index = False)

# Putting all the Non Twitter Data in One Frame

In [171]:
df0 = pd.read_csv("2010_examples_teams.csv")
df1 = pd.read_csv("2011_examples_teams.csv")
df2 = pd.read_csv("2012_examples_teams.csv")
df = pd.concat([df0,df1,df2])
df

,Home Scoring,Home Pass Yds,Home Rush Yds,Home Fumbles,Home Ints,Away Scoring,Away Pass Yds,Away Rush Yds,Away Fumbles,Away Ints,Home Field Adv,Home Team,Away Team,Week,Target,Unnamed: 0
0,16.500000,209.500000,101.500000,2.500000,0.500000,24.000000,253.000000,83.500000,1.000000,0.500000,0.000000,cin,rav,2,1,NaN
1,23.500000,203.000000,131.500000,1.000000,0.500000,14.000000,208.000000,139.500000,2.000000,1.000000,0.000000,oti,pit,2,0,NaN
2,21.000000,185.000000,132.000000,0.000000,1.000000,15.000000,169.000000,99.500000,2.000000,0.000000,1.000000,gnb,buf,2,1,NaN
3,20.500000,179.500000,76.000000,1.500000,1.500000,19.500000,285.500000,125.000000,3.500000,0.500000,1.000000,det,phi,2,0,NaN
4,14.000000,267.000000,106.500000,3.000000,1.000000,19.000000,125.000000,127.000000,1.000000,0.500000,1.000000,cle,kan,2,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,24.928571,254.857143,122.071429,1.214286,0.714286,20.785714,260.571429,103.892857,1.392857,0.821429,0.071429,car,rai,16,1,219.0
220,25.678571,274.964286,115.392857,1.500000,0.928571,19.785714,235.964286,94.214286,1.428571,1.178571,0.142857,den,cle,16,1,220.0
221,16.571429,218.071429,94.500000,1.785714,1.392857,22.250000,211.000000,128.035714,1.071429,1.357143,0.071429,crd,chi,16,0,221.0
222,23.285714,260.107143,114.392857,1.357143,1.071429,24.285714,257.178571,123.607143,1.250000,1.000000,0.000000,rav,nyg,16,1,222.0


In [172]:
df = df.drop(columns=['Home Team',"Away Team","Week","Unnamed: 0"])

In [173]:
df

,Home Scoring,Home Pass Yds,Home Rush Yds,Home Fumbles,Home Ints,Away Scoring,Away Pass Yds,Away Rush Yds,Away Fumbles,Away Ints,Home Field Adv,Target
0,16.500000,209.500000,101.500000,2.500000,0.500000,24.000000,253.000000,83.500000,1.000000,0.500000,0.000000,1
1,23.500000,203.000000,131.500000,1.000000,0.500000,14.000000,208.000000,139.500000,2.000000,1.000000,0.000000,0
2,21.000000,185.000000,132.000000,0.000000,1.000000,15.000000,169.000000,99.500000,2.000000,0.000000,1.000000,1
3,20.500000,179.500000,76.000000,1.500000,1.500000,19.500000,285.500000,125.000000,3.500000,0.500000,1.000000,0
4,14.000000,267.000000,106.500000,3.000000,1.000000,19.000000,125.000000,127.000000,1.000000,0.500000,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
219,24.928571,254.857143,122.071429,1.214286,0.714286,20.785714,260.571429,103.892857,1.392857,0.821429,0.071429,1
220,25.678571,274.964286,115.392857,1.500000,0.928571,19.785714,235.964286,94.214286,1.428571,1.178571,0.142857,1
221,16.571429,218.071429,94.500000,1.785714,1.392857,22.250000,211.000000,128.035714,1.071429,1.357143,0.071429,0
222,23.285714,260.107143,114.392857,1.357143,1.071429,24.285714,257.178571,123.607143,1.250000,1.000000,0.000000,1


In [174]:
df.to_csv("all_years.csv",index = False)